In [1]:
from dotenv import load_dotenv
import os

load_dotenv("keys.env") 
ACTIVELOOP_TOKEN= os.getenv("ACTIVELOOP_TOKEN")
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID= os.getenv("GOOGLE_CSE_ID")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import Tool
from duckduckgo_search import DDGS
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

from langchain_community.tools.tavily_search import TavilySearchResults
tavily_tool = TavilySearchResults(max_results=5)
def duckduckgo_search(query: str):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=5)
    return "\n".join([result["title"] + ": " + result["href"] for result in results])
#Set up the tools
search = GoogleSearchAPIWrapper()
tools = [
     tavily_tool,
    WriteFileTool(),
    ReadFileTool(),
]


C:\Users\akash\AppData\Local\Temp\ipykernel_27548\2852505894.py:14: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchAPIWrapper``.
  search = GoogleSearchAPIWrapper()


In [3]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
import ollama

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embedding_size = 384 


C:\Users\akash\AppData\Local\Temp\ipykernel_27548\2755117910.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\akash\anaconda3\envs\Project_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

import faiss
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embedding_function=embeddings.embed_query, index=index, docstore=InMemoryDocstore({}), index_to_docstore_id={})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [5]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI
from langchain_ollama import ChatOllama

llm_engine=ChatOllama(
    model="llama3.1",
    temperature=5
)
agent = AutoGPT.from_llm_and_tools(
    ai_name="Jim",
    ai_role="Assistant",
    tools=tools,
    llm=llm_engine,
    memory=vectorstore.as_retriever()
)

# Set verbose to be true
agent.chain.verbose = True


In [ ]:
task = "what was indias role in world war 2"

agent.run([task])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. what was indias role in world war 2


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query., args json sch

Token indices sequence length is longer than the specified maximum sequence length for this model (1156 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. what was indias role in world war 2


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. tavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query., args json sch

In [ ]:
relevant_docs = memory.invoke(str(previous_messages[-10:]))
